In [110]:
# Service requests require credentials
#     Google has classified service request from local machine as server-to-server.
#     I have downloaded credentials as a json file to local machine. Here, I create
#     a credentials object by supplying the downloaded file.
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/cloud-platform']
SERVICE_ACCOUNT_FILE = 'vision_fission.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Also note: From https://console.developers.google.com/
# 1. A capsule of work is a project.
# 2. Services need to be enabled for a project.
# 3. Billing needs to be enabled for a project with valid Credit Card.
# Else, NO GO!

# Ref: https://google-cloud-python.readthedocs.io/en/latest/vision/index.html

In [111]:
# Create a client object for a specific service and
# pass it credentials. Ensure that you have satisfied
# all conditions noted above. 
# Service methods are invoked using the client object.
import io
import os
from google.cloud import vision
from google.cloud.vision import types

client = vision.ImageAnnotatorClient(
    credentials=credentials
)

# Ref: https://google-cloud-python.readthedocs.io/en/latest/vision/index.html

In [112]:
# There is the triumvirate of 
# 1. Input (image file)
# 2. Service (vision client)
# 3. Output (json)
# Ingest the image file with io
file_name = 'sunbeam.JPG'

with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Ref: https://cloud.google.com/vision/docs/libraries#client-libraries-install-python

In [ ]:
# Make the service call and see what's in the response.
response = client.label_detection(image=image)
response

In [ ]:
response = client.annotate_image({
    'image': image, #{'source': {'image_uri': '.\my-test-bucket\sunbeam.jpg'}},
    'features': [{'type': vision.enums.Feature.Type.FACE_DETECTION}],
})
response

In [ ]:
def show_results(inputfile, data, outputfile):
    #read original file
    im = read_image(inputfile)
    
    #draw face, boxes and text for each response
    for r in data['responses']:
        if 'faceAnnotations' in r:
            draw_face(im, r['faceAnnotations'])
        
        if 'labelAnnotations' in r:
            strs = map(lambda a: a['description'], r['labelAnnotations'])
            im = draw_text(im, ", ".join(strs))
            
        for field in ['textAnnotations', 'logoAnnotations']:
            if field in r:
                for a in r[field]:
                    draw_box(im, a['boundingPoly']['vertices'])

    #save to output file
    save_image(outputfile, im)